# Test compatibility gurobi
The idea is test if the model development is compatible with gurobi constraints. Using the packaege gurobi machine learning

**TESTING MODELS WITH GUROBI LICENCE** To test more complex models that using a lot of parameters and needs the licence to test if they are accepted by gurobi

## Root folder and read env variables

In [1]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  D:\github-mi-repo\Optimization-Industrial-Process-Advanced


In [2]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## LOAD LICENCE GUROBI

In [3]:
##########  LOAD LICENCE GUROBI ##########
import gurobipy as gp

# set env variable with the path of the licence
name_file_licence_gurobi = "gurobi.lic"
path_licence_gurobi = root_path + '\\' + name_file_licence_gurobi
os.environ ["GRB_LICENSE_FILE"] = path_licence_gurobi
print(os.environ["GRB_LICENSE_FILE"])

D:\github-mi-repo\Optimization-Industrial-Process-Advanced\gurobi.lic


In [4]:
######### LAOD CONTENT LICENCE GUROBI #########
with open(path_licence_gurobi, 'r') as f:
    content_licence = f.read()
WLSACCESSID = content_licence.split('\n')[3].split('=')[1] # load WLSACCESSID (string)
WLSSECRET = content_licence.split('\n')[4].split('=')[1] # load WLSSECRET (string)
LICENSEID = int(content_licence.split('\n')[5].split("=")[1]) # load LICENSEID (integer)

params = {
"WLSACCESSID": WLSACCESSID,
"WLSSECRET": WLSSECRET,
"LICENSEID": LICENSEID
}

In [5]:
######### lOAD GUROBI MODEL #########
env = gp.Env(params=params)

#Create the model within the Gurobi environment
m = gp.Model("Gurobi test model")
m

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2441807
WLS license 2441807 - registered to CMPC Celulosa S.A
Restricted license - for non-production use only - expires 2025-11-24


<gurobi.Model Continuous instance Gurobi test model: 0 constrs, 0 vars, No parameter changes>

## RUN

In [6]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
import gcsfs
import pickle

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

# transform
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer

# models
from sklearn.linear_model import LinearRegression # lr
from sklearn.linear_model import Ridge # ridge
from sklearn.linear_model import Lasso # lasso
from sklearn.tree import DecisionTreeRegressor # tree
from sklearn.ensemble import GradientBoostingRegressor #gb
from sklearn.ensemble import RandomForestRegressor #rf
#from xgboost import XGBRegressor # xgb
from  sklearn.neural_network import MLPRegressor # mlp

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

### 0. Load data
This data will be use to get values to generate a instance of the ml model

In [7]:
path_data = 'artifacts/data/data.pkl'
data = pd.read_pickle(path_data)
data.head(3)

,230AIT446.PNT,240AIC022.MEAS,240AIC126.MEAS,240AIC224.MEAS,240AIC286.MEAS,240AIC324.MEAS,240AIC433.MEAS,240AIT063A.PNT,240AIT063B.PNT,240AIT225A.PNT,...,S240ALDP022,S240ALDP031,S240ALDP032,S276PER002,S2MAQUINAT07,S76ALE017,SSTRIPPING015,calc_prod_d0,calc_prod_d1,calc_prod_p
datetime,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:05:00,11.555040,2.983948,11.346645,4.413519,4.352375,10.441675,4.292521,5.86932,62.37495,1.837519,...,91.49,1.8,11.4,11.77,1.5712,173.6,964.0,3240.8635,3313.6215,3259.3745
2021-01-01 00:10:00,11.552320,3.015669,11.353215,4.413179,4.347186,10.432170,4.289684,5.86932,62.37495,1.814020,...,91.49,1.8,11.4,11.77,1.5712,173.6,964.0,3260.7475,3301.6920,3208.6785
2021-01-01 00:15:00,11.549955,3.018903,11.355525,4.408321,4.355828,10.410115,4.284427,5.86932,62.37495,1.814020,...,91.49,1.8,11.4,11.77,1.5712,173.6,964.0,3265.5765,3284.1330,3210.7790


### 1. Load Artifacts to connect ML to gurobi

#### 1.1 pkl model

In [8]:
# define function euler negative clo2
def euler_dioxido(values):
    factor_a = 1/10
    output = np.power(np.e, -factor_a * values)
    return output

In [9]:
path_model_to_test = f'artifacts/models/d0eop_microkappa/model.pkl'
model_ml_to_test = pd.read_pickle(path_model_to_test)
model_ml_to_test

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/models/d0eop_microkappa/model.pkl'

### 1.2 Define list of features and target for each model

In [ ]:
######################## model d0eop_microkappa ########################

list_features_d0eop_microkappa = [
    "240AIT063A.PNT", #kappa_d0
    "240AIT063B.PNT", #brillo_d0
    "calc_prod_d0", #calc_prod_d0
    "240FY050.RO02", #especifico_dioxido_d0 - VC
    "SSTRIPPING015", #dqo_evaporadores
    "S276PER002", #concentracion_clo2_d0
    "240AIC022.MEAS", #ph_a
    "240FY118B.RO01", #especifico_oxigeno_eop - VC
    "240FY11PB.RO01", #especifico_peroxido_eop - VC
    "240FY107A.RO01", #especifico_soda_eop - VC
]

list_features_controlables_d0eop_microkappa = [
    "240FY050.RO02", #especifico_dioxido_d0 - VC,
    "240FY118B.RO01", #especifico_oxigeno_eop - VC
    "240FY11PB.RO01", #especifico_peroxido_eop - VC
    "240FY107A.RO01", #especifico_soda_eop - VC
]

list_target_d0eop_microkappa = ['240AIT225A.PNT'] # microkappa_d1

### 1.3 Read master tag and sort features according its order

In [ ]:
# read table master tag
name_model = 'd0eop_microkappa'
path_list_features_target_to_optimization = f'config/optimization_engine/ml_models/MaestroTags-{name_model}-general.xlsx'
maestro_tags = pd.read_excel(path_list_features_target_to_optimization)

### sort list of features according the order in master table
list_features_d0eop_microkappa = [tag for tag in maestro_tags['TAG'].tolist() if tag in list_features_d0eop_microkappa]
list_features_controlables_d0eop_microkappa = [tag for tag in maestro_tags['TAG'].tolist() if tag in list_features_controlables_d0eop_microkappa]

## 2. Create gurobi model

In [ ]:
# create model
m = gp.Model('modelo')

### 3. Create decision variables
- Decision variables that are features in ml models
- Decicion variable that is the output in ml models

In [ ]:
# define set
list_bleaching = ['bleaching']
index_bleaching = pd.Index(list_bleaching)
index_bleaching

In [ ]:
# create decision variables - features ml model
especifico_dioxido_d0 = gppd.add_vars(m, index_bleaching, name = "especifico_dioxido_d0"
                                     )

especifico_oxigeno_eop = gppd.add_vars(m, index_bleaching, name = "especifico_oxigeno_eop"
                                     )

especifico_peroxido_eop = gppd.add_vars(m, index_bleaching, name = "especifico_peroxido_eop"
                                     )

especifico_soda_eop = gppd.add_vars(m, index_bleaching, name = "especifico_soda_eop"
                                     )

In [ ]:
# crete decision variables - output ml model
microkappa_d1 = gppd.add_vars(m, index_bleaching, name = "especifico_dioxido_d0"
                                     )

In [ ]:
# "compile"
m.update()

In [ ]:
# see decision var created
especifico_soda_eop

### 4. Create instance of Machine learning model using decision var of gurobi (decision var in optimization)

In [ ]:
######################## generate instance NO controlables features for model d0eop_microkappa ########################

# list feature NC
list_features_d0eop_microkappa_no_vc = list(set(list_features_d0eop_microkappa) - set(list_features_controlables_d0eop_microkappa))

# generate dataframe with the mean
instance_no_controlables_d0eop_microkappa = data[list_features_d0eop_microkappa_no_vc].mean().to_frame().T
instance_no_controlables_d0eop_microkappa

In [ ]:
######################## genrate instance - features no controlables + decision vars ########################

# create instance with controlables variables. sorted according the list of features. ES MUY IMPORTANTE QUE ESTÉ ORDENADO LAS VARIABLES DE DECUISIÓN DE ACUERDO A LA LISTA DE FEATURES
instance_controlables_d0eop_microkappa = pd.DataFrame([especifico_dioxido_d0, especifico_oxigeno_eop, especifico_peroxido_eop, especifico_soda_eop]).T
instance_controlables_d0eop_microkappa.columns = list_features_controlables_d0eop_microkappa # rename columns
instance_controlables_d0eop_microkappa.reset_index(inplace = True)
instance_controlables_d0eop_microkappa.drop(columns = 'index', inplace = True)

# append features controlables with no controlables
instance_d0eop_microkappa = pd.concat([instance_no_controlables_d0eop_microkappa, instance_controlables_d0eop_microkappa], axis = 1)
instance_d0eop_microkappa = instance_d0eop_microkappa[list_features_d0eop_microkappa] # sort features

# set index - optimization set
instance_d0eop_microkappa.index = index_bleaching

In [ ]:
instance_d0eop_microkappa

In [ ]:
###### load ml constraint ######
pred_constr_d0eop_microkappa = add_predictor_constr(gp_model = m, 
                                                    predictor = model_ml_to_test, 
                                                    input_vars = instance_d0eop_microkappa, 
                                                    output_vars = microkappa_d1,
                                                    name = f'model_predict_d0eop_microkappa'
                                                   )
pred_constr_d0eop_microkappa.print_stats()

#### NOTE IN THIS PART YOU CAN SEE IF THE MODEL CAN CONNNECT TO GUROBI!

### 5. Define objective optimization
Objetive that no generate infeasibility

In [ ]:
m.setObjective(microkappa_d1.sum(),
               gp.GRB.MINIMIZE)

#### 6. Optimize and get optimal values

In [ ]:
# solve
m.optimize()

In [ ]:
#### know the status of the model - 2 a optimal solution was founded
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
m.Status

In [ ]:
# get optimal values and save in a dataframe
######## create a dataframe with set as index
solution = pd.DataFrame(index = index_bleaching)

######################## save optimal values - features of models (only the features) ########################

# model d0eop_microkappa
solution["especifico_dioxido_d0"] = especifico_dioxido_d0.gppd.X
solution["especifico_oxigeno_eop"] = especifico_oxigeno_eop.gppd.X
solution["especifico_peroxido_eop"] = especifico_peroxido_eop.gppd.X
solution["especifico_soda_eop"] = especifico_soda_eop.gppd.X

######################## save optimal values - targets of models (some targets are features of the model of the next step) ########################
solution["microkappa_d1"] = microkappa_d1.gppd.X  # model d0eop_microkappa


######################## # get value objetive function ########################
opt_objetive_function = m.ObjVal

In [ ]:
# show value objetive function
opt_objetive_function

In [ ]:
# show value decision variables
solution